## This  is a notebook

In [61]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import Imputer
from sklearn.metrics import mean_absolute_error

In [68]:
properties = pd.read_csv('../data/properties_2016.csv')
train_data = pd.read_csv('../data/train_2016_v2.csv')
template_submission = pd.read_csv('../data/sample_submission.csv')

C:\Program Files\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## Preprocessing

- merge data frames
- remove crappy features (missing > .75)
- make sure the types make sense


we have data of houses that have been sold, those have a logerror. This logerror is our train_y. Split data set on logerror / no logerror. The other is test?



In [72]:
template_submission.shape

(2985217, 7)

In [71]:
properties.shape

(2985217, 58)

In [30]:
merged_df = pd.merge(train_data, properties,on='parcelid')

In [78]:
temp_df = merged_df[['bathroomcnt', 'bedroomcnt', 'longitude', 'latitude','logerror']].dropna()
properties_filtered = properties[['bathroomcnt', 'bedroomcnt', 'longitude', 'latitude']]
properties_filtered = properties_filtered.fillna(properties_filtered.mean())
X = temp_df[['bathroomcnt', 'bedroomcnt', 'longitude', 'latitude']]
Y = temp_df.logerror

In [80]:
y_train = Y[:80000]
y_test = Y[80000:]
x_train = X[:80000]
x_test = X[80000:]

x_submission = properties_filtered

## Feature engineering

In [24]:
feature_engineering = Imputer()

## Model

In [20]:
model = LinearRegression()

## Pipeline

In [49]:
pipe = Pipeline([#('feature_engineering', feature_engineering),
                 ('model', model)
                 ]
)

Fit model

In [56]:
pipe.fit(x_train, y_train)

Pipeline(memory=None,
     steps=[('model', LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False))])

Predict on test

In [63]:
y_pred = pipe.predict(x_test)

In [64]:
result = mean_absolute_error(y_test, y_pred)

In [65]:
result

0.066561026013171654

In [81]:
out_submission = pipe.predict(x_submission)

In [88]:
out_formatted = pd.DataFrame([template_submission.ParcelId, out_submission, out_submission, out_submission, out_submission], columns=template_submission.columns.values)

In [103]:
out_formatted = template_submission

In [108]:
for col in out_formatted.columns.values:
    if col == 'ParcelId':
        pass
    else:
        out_formatted[col] = out_submission
out_formatted.head()

,ParcelId,201610,201611,201612,201710,201711,201712
0,10754147,-0.001635,-0.001635,-0.001635,-0.001635,-0.001635,-0.001635
1,10759547,-0.001691,-0.001691,-0.001691,-0.001691,-0.001691,-0.001691
2,10843547,-0.002399,-0.002399,-0.002399,-0.002399,-0.002399,-0.002399
3,10859147,-0.001984,-0.001984,-0.001984,-0.001984,-0.001984,-0.001984
4,10879947,-0.001971,-0.001971,-0.001971,-0.001971,-0.001971,-0.001971


In [110]:
out_formatted.to_csv('../data/submission_1_JFR')